<a href="https://colab.research.google.com/github/chelsea-riwash/Weather-prediction-and-Forecasting/blob/main/Weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #To convert catagorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #model for classification and regression
from sklearn.metrics import mean_squared_error #to measure accuracy of our prediction
from datetime import datetime, timedelta #to handle date and time
import pytz

In [27]:
API_KEY = '3e961e4f5b93c760b8d1eaecce7ac543'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

**1. Fetch Current Weather Data**

In [28]:
def get_current_weather(city):
    # Make the API call using the city name
    response = requests.get(f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric")

    # Parse the response data
    data = response.json()

    if response.status_code == 200:
        # Parse the relevant data from the response
        weather_data = {
            'temp_min': data['main']['temp_min'],
            'temp_max': data['main']['temp_max'],
            'current_temp': data['main']['temp'],
            'feels_like': data['main']['feels_like'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            'wind_gust_dir': data['wind']['deg'],
            'Wind_Gust_Speed': data['wind']['speed'],
            'description': data['weather'][0]['description'],
            'country': data['sys']['country'],
        }
        return weather_data
    else:
        print(f"Error: {data['message']}")
        return None

**2. Read Historical Data**

In [29]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

**3. Prepare data for training**


In [30]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

#feature variable and target variables
  X = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']] #feature variables
  y = data['RainTomorrow'] #target variable

  return X,y,le

**4. Train Rain Prediction Model**

In [31]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model

  y_pred = model.predict(X_test) #to make predictions on test set

  print(mean_squared_error(y_test, y_pred))

  return model

**5. Prepare regression data**

In [32]:
def prepare_regression_data(data, feature):
  X, y = [],[] #initialize list for feature and target values

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X=np.array(X).reshape(-1,1)
  y = np.array(y)
  return X, y

**6. Train Regression Model**

In [33]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X,y)
  return model

**7. Predict Future**

In [34]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]

**8. Weather Analysis Function**

In [39]:
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)

  #load historical data
  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model

  X, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(X, y)

  #map wind direction to compass points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
        ("N", 0, 22.5),
        ("NNE", 22.5, 45),
        ("NE", 45, 67.5),
        ("ENE", 67.5, 90),
        ("E", 90, 112.5),
        ("ESE", 112.5, 135),
        ("SE", 135, 157.5),
        ("SSE", 157.5, 180),
        ("S", 180, 202.5),
        ("SSW", 202.5, 225),
        ("SW", 225, 247.5),
        ("WSW", 247.5, 270),
        ("W", 270, 292.5),
        ("WNW", 292.5, 315),
        ("NW", 315, 337.5),
        ("NNW", 337.5, 360)
    ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg <end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp':current_weather['temp_min'],
      'MaxTemp':current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['Wind_Gust_Speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp':  current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for temperature and humidity
  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')

  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)

  hum_model = train_regression_model(X_hum, y_hum)

  #predict future temperature and humidity
  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])
  timezone = pytz.timezone('Asia/Kathmandu')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #Display Results
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Prediction:")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Prediction:")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter any city name: nepalgunj
0.1506849315068493
City: nepalgunj, NP
Current Temperature: 21.26
Feels Like: 20.02
Minimum Temperature: 21.26°C
Maximum Temperature: 21.26°C
Humidity: 22%
Weather Prediction: clear sky
Rain Prediction: No

Future Temperature Prediction:
11:00: 20.5°C
12:00: 22.2°C
13:00: 22.7°C
14:00: 25.0°C
15:00: 26.7°C

Future Humidity Prediction:
11:00: 35.1%
12:00: 38.4%
13:00: 52.3%
14:00: 43.4%
15:00: 46.8%
